In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, random
# %matplotlib inline

import torch
from torch import nn
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用

from importlib import reload 
from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

df=pd.DataFrame()  # 初始化空值
# pd.options.display.float_format = '{:.7g}'.format 
vTest = 1000

import torch.backends.cudnn as cudnn
cudnn.benchmark = True  
cudnn.deterministic = True

xl_01dgree = 0.001745 # 0.1度 
xl_02dgree = 0.003490 # 0.2度 
xl_03dgree = 0.005230 # 0.3度
xl_dgree = xl_03dgree

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print( device )
# if torch.cuda.is_available():
#     torch.cuda.caching_allocator_delete
#     torch.cuda.empty_cache()  # 释放显存
#     print('Memory Allocated', torch.cuda.memory_allocated() )

cuda:0


In [2]:
sql="SELECT * FROM ds_qf60"
# sql="SELECT * FROM ds_qf60_2"
df = pd.read_sql_query(sql, conn, index_col=None)
del df['index']

In [3]:
""" 确定训练的目标 """
# df['class0'] = df['test5_8']
df['class0'] = df['test10_10']
# df['class0'] = df['test20_20']
# df['class0'] = df['test20_30']
df.info(), df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292325 entries, 0 to 292324
Data columns (total 24 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   pct_change    292325 non-null  float64
 1   diff          292325 non-null  float64
 2   dea           292325 non-null  float64
 3   bar           292325 non-null  float64
 4   jx_days_ud60  292325 non-null  int64  
 5   jx_xl_250     292325 non-null  float64
 6   jx_xl_120     292325 non-null  float64
 7   jx_xl_60      292325 non-null  float64
 8   jx_xl_20      292325 non-null  float64
 9   jx_xl_10      292325 non-null  float64
 10  jx_xl_5       292325 non-null  float64
 11  jx_zs_5       292325 non-null  int64  
 12  jx_zs_10      292325 non-null  int64  
 13  jx_zs_20      292325 non-null  int64  
 14  jx_zs_60      292325 non-null  int64  
 15  lj_fl_1_3     292325 non-null  float64
 16  lj_fl_3_10    292325 non-null  float64
 17  lj_fl_5_20    292325 non-null  float64
 18  lj_f

(None, (292325, 24))

In [4]:
df.class0.unique(), df.class0.value_counts()

(array([0, 1], dtype=int64),
 0    246721
 1     45604
 Name: class0, dtype: int64)

In [5]:
Y_data = df.class0.values.reshape(-1,1)  # 转换成 pd type array
Y = torch.from_numpy(Y_data).type(torch.float32)
Y.shape, Y.size(0)

(torch.Size([292325, 1]), 292325)

In [6]:
X_columns = ['pct_change', 
             'diff', 'dea', 'bar', 
             'jx_days_ud60', 
             'jx_xl_250', 'jx_xl_120','jx_xl_60', 
             'jx_xl_20', 'jx_xl_10', 'jx_xl_5', 
             'jx_zs_5', 'jx_zs_10','jx_zs_20', 'jx_zs_60', 
             'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20','lj_fl_20_60']
X_data = df[X_columns]  
X = torch.from_numpy(X_data.values).type(torch.float32)
X.shape, X.size()

(torch.Size([292325, 19]), torch.Size([292325, 19]))

In [7]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(19, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.sigmoid( self.liner_3(x) )
        return x

class Model4A(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(19, 128)   # X.size(1)= 20； 定义256个后续中间层
        self.liner_2 = nn.Linear(128, 128)  # 接着，再定义128个后续中间层
        self.liner_3 = nn.Linear(128, 32)   # 接着，再定义32个后续中间层
        self.liner_4 = nn.Linear(32, 1)     
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.relu( self.liner_3(x) )
        x = F.sigmoid( self.liner_4(x) )
        return x
    
class Model4B(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(19, 128)   # X.size(1)= 20； 定义256个后续中间层
        self.liner_2 = nn.Linear(128, 128)  # 接着，再定义128个后续中间层
        self.liner_3 = nn.Linear(128, 128)  # 接着，再定义128个后续中间层
        self.liner_4 = nn.Linear(128, 32)   # 接着，再定义32个后续中间层
        self.liner_5 = nn.Linear(32, 1)     
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.relu( self.liner_3(x) )
        x = F.relu( self.liner_4(x) )
        x = F.sigmoid( self.liner_5(x) )
        return x

# 更加高级语言法，等价于Model
class Model_high(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(29, 256)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(256, 128)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(128, 64)   # 接着，再定义64个后续中间层
        self.liner_4 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.Liner_1(input)
        x = self.relu(x)
        x = self.Liner_2(x)
        x = self.relu(x)
        x = self.Liner_3(x)
        x = self.sigmoid(x)
        return x

In [8]:
# Ver 4.2 - sklearn mode

lr = 0.00012
def get_model():
    model = Model4B()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    return model, opt

model, optim = get_model()
model = model.to(device)
# model, optim 

loss_fn = nn.BCELoss()
loss_fn = loss_fn.to(device)
# batch = 64
# batch = 1024
batch = 8192
# batch = 16384
# batch = 32768
epochs = 10
no_of_batches = len(X)//batch
from torch.utils.data import TensorDataset, DataLoader

def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).type(torch.int32)
    acc = (y_pred == y_true).float().mean()
    return acc

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)

train_x = torch.from_numpy(train_x.values).type(torch.float32).to(device)
train_y = torch.from_numpy(train_y).type(torch.float32).to(device)
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

test_x = torch.from_numpy(test_x.values).type(torch.float32).to(device)
test_y = torch.from_numpy(test_y).type(torch.float32).to(device)
test_ds = TensorDataset(test_x, test_y)
test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量

train_x.shape, test_x.shape, train_y.shape, test_y.shape

(torch.Size([219243, 19]),
 torch.Size([73082, 19]),
 torch.Size([219243, 1]),
 torch.Size([73082, 1]))

计算正确率
- Sigmoid()是0和1之间的值， 大于0.5为1，否则是0
- out1 = (y-pred >0.5).type(torch.int32)  : 转换成0或1
- (out1 == labels).float().mean()

In [9]:
%%time
# epochs = 10
for epoch in range( epochs ):
    for x,y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    with torch.no_grad():
        epoch_accuracy = accuracy(model(train_x), train_y)
        epoch_loss = loss_fn(model(train_x), train_y).data.item()
        epoch_test_accuracy = accuracy(model(test_x), test_y)
        epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
        print('epoch: ', 
            epoch, ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
            epoch, ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

epoch:  0  train_loss:  0.5617  Acc： 0.8443 0  test_loss:  0.562  Acc： 0.8432
epoch:  1  train_loss:  0.4672  Acc： 0.8443 1  test_loss:  0.4679  Acc： 0.8432
epoch:  2  train_loss:  0.4342  Acc： 0.8443 2  test_loss:  0.4355  Acc： 0.8432
epoch:  3  train_loss:  0.4335  Acc： 0.8443 3  test_loss:  0.4349  Acc： 0.8432
epoch:  4  train_loss:  0.4328  Acc： 0.8443 4  test_loss:  0.4341  Acc： 0.8432
epoch:  5  train_loss:  0.4318  Acc： 0.8443 5  test_loss:  0.4331  Acc： 0.8432
epoch:  6  train_loss:  0.4307  Acc： 0.8443 6  test_loss:  0.432  Acc： 0.8432
epoch:  7  train_loss:  0.4296  Acc： 0.8442 7  test_loss:  0.4309  Acc： 0.8431
epoch:  8  train_loss:  0.4288  Acc： 0.8441 8  test_loss:  0.4299  Acc： 0.8431
epoch:  9  train_loss:  0.4279  Acc： 0.844 9  test_loss:  0.4291  Acc： 0.843
CPU times: total: 9.45 s
Wall time: 29.6 s


In [18]:
accuracy(model(test_x), test_y)

tensor(0.8430, device='cuda:0')

In [11]:
train_m, test_m, train_n, test_n = train_test_split(X_data, Y_data, shuffle=True)

# train_x = torch.from_numpy(train_x.values).type(torch.float32).to(device)
# train_y = torch.from_numpy(train_y).type(torch.float32).to(device)
# train_ds = TensorDataset(train_x, train_y)
# train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

test_m = torch.from_numpy(test_m.values).type(torch.float32).to(device)
test_n = torch.from_numpy(test_n).type(torch.float32).to(device)
accuracy(model(test_m), test_n)

tensor(0.8441, device='cuda:0')

In [12]:
""" 模型保存 """
# torch.save( model, '../model/test5-8.pth')
# path = '../model/test5-8'
# torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optim.state_dict(),
#            }, path)

' 模型保存 '

In [13]:
# list1 = []
# for i in range(0,X_app.size(0)):
#     list1.append( random.randint(1,1) )
# len(list1)
# Y_app = torch.tensor( list1 )
# Y_app.to(device)
# Y_app.reshape(-1,1).size(0) 

In [47]:
""" 手工取数据: 测试(1) """
column_str = 'ts_code, trade_date, pct_change, diff, dea, bar, jx_days_ud60, \
jx_xl_250, jx_xl_120, jx_xl_60,\
jx_xl_20, jx_xl_10, jx_xl_5, \
jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,\
lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, \
sz_max_5, sz_max_10, sz_max_20 '
# print(column_str)

sql_new= "SELECT " + column_str +" \
from day_zhibiao \
where trade_date>='20230601' and sz_max_10>10 and \
jx_days_ud60 > 20 and jx_days_ud60 < 60 and \
jx_xl_60>-"+str(xl_dgree)+" and  \
jx_xl_20>0 "
print(sql_new)
df_new = pd.read_sql_query(sql_new, conn)
print( df_new.shape )
# print( df_new[ df_new['sz_max_5']>8 ].head(3) )
print( df_new[ df_new['sz_max_10']>10 ][['ts_code','trade_date','sz_max_10','sz_max_20']].head(3) )

X_columns = ['pct_change', 
             'diff', 'dea', 'bar', 
             'jx_days_ud60', 
             'jx_xl_250', 'jx_xl_120','jx_xl_60', 
             'jx_xl_20', 'jx_xl_10', 'jx_xl_5', 
             'jx_zs_5', 'jx_zs_10','jx_zs_20', 'jx_zs_60', 
             'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20','lj_fl_20_60']
df_app = df_new[X_columns]
X_app = torch.from_numpy(df_app.values).type(torch.float32).to(device)

# print( df_app.head(3) )
print('X_app: ', X_app.shape, X_app.size )

SELECT ts_code, trade_date, pct_change, diff, dea, bar, jx_days_ud60, jx_xl_250, jx_xl_120, jx_xl_60,jx_xl_20, jx_xl_10, jx_xl_5, jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, sz_max_5, sz_max_10, sz_max_20  from day_zhibiao where trade_date>='20230601' and sz_max_10>10 and jx_days_ud60 > 20 and jx_days_ud60 < 60 and jx_xl_60>-0.00523 and  jx_xl_20>0 
(651, 24)
     ts_code trade_date  sz_max_10  sz_max_20
0  000510.SZ   20230613    11.6554    11.6554
1  000510.SZ   20230612    11.2795    11.2795
2  000561.SZ   20230602    14.5766    24.0086
X_app:  torch.Size([651, 19]) <built-in method size of Tensor object at 0x0000019A893D0A40>


In [43]:
list1 = []
for i in range(0,X_app.size(0)):
    list1.append( 1.0 )
Y_app = torch.tensor( list1).type(torch.float32).reshape(-1,1).to(device)
Y_app.shape


torch.Size([651, 1])

In [46]:
# Y_app = torch.tensor( list1).type(torch.float32)
X_app.shape, Y_app.shape


(torch.Size([651, 19]), torch.Size([651, 1]))

In [ ]:
print( '准确率：', accuracy(model(X_app), Y_app) )

In [ ]:
test_y
Y_app

In [31]:
# X_app
# Y_app.to(device)
accuracy(model(X_app), Y_app)
# accuracy(model(test_x), test_y)

tensor(0., device='cuda:0')

In [34]:
""" 手工取数据: 测试 (2) """
def model_app(X_app):
    Y_app = model(X_app)
    Y_app2 = Y_app.to('cpu').detach()
    y_app = Y_app2.numpy() 
    acc_num =  np.argwhere(y_app>0.5)
    return Y_app2, acc_num

column_str = 'ts_code, trade_date, pct_change, diff, dea, bar, jx_days_ud60, \
jx_xl_250, jx_xl_120, jx_xl_60,\
jx_xl_20, jx_xl_10, jx_xl_5, \
jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,\
lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, \
sz_max_5, sz_max_10, sz_max_20 '
# print(column_str)

sql_new= "SELECT " + column_str +" \
from day_zhibiao \
where trade_date>='20230601' and sz_max_10>10 and \
jx_days_ud60 > 20 and jx_days_ud60 < 60 and \
jx_xl_60>-"+str(xl_dgree)+" and  \
jx_xl_20>0 "
print(sql_new)
df_new = pd.read_sql_query(sql_new, conn)
print( df_new.shape )
# print( df_new[ df_new['sz_max_5']>8 ].head(3) )
print( df_new[ df_new['sz_max_10']>10 ].head(3) )

X_columns = ['pct_change', 
             'diff', 'dea', 'bar', 
             'jx_days_ud60', 
             'jx_xl_250', 'jx_xl_120','jx_xl_60', 
             'jx_xl_20', 'jx_xl_10', 'jx_xl_5', 
             'jx_zs_5', 'jx_zs_10','jx_zs_20', 'jx_zs_60', 
             'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20','lj_fl_20_60']
df_app = df_new[X_columns]
X_app = torch.from_numpy(df_app.values).type(torch.float32).to(device)
# print( df_app.head(3) )
print('X_app: ', X_app.shape, X_app.size() )

Y_app2, acc_num = model_app(X_app) 
print('找出数量： ', acc_num )

SELECT ts_code, trade_date, pct_change, diff, dea, bar, jx_days_ud60, jx_xl_250, jx_xl_120, jx_xl_60,jx_xl_20, jx_xl_10, jx_xl_5, jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, sz_max_5, sz_max_10, sz_max_20  from day_zhibiao where trade_date>='20230601' and sz_max_10>10 and jx_days_ud60 > 20 and jx_days_ud60 < 60 and jx_xl_60>-0.00523 and  jx_xl_20>0 
(651, 24)
     ts_code trade_date  pct_change      diff       dea       bar  \
0  000510.SZ   20230613     -0.3367 -0.048761 -0.067651  0.037780   
1  000510.SZ   20230612      2.5907 -0.062555 -0.072373  0.019636   
2  000561.SZ   20230602      0.8649  0.023673 -0.026637  0.100619   

   jx_days_ud60  jx_xl_250  jx_xl_120  jx_xl_60  ...  jx_zs_10  jx_zs_20  \
0            47  -0.000142   0.000122 -0.001376  ...         3         2   
1            46  -0.000129   0.000122 -0.000825  ...         2         1   
2            47   0.001256   0.000402 -0.002363  ...         3        13   

   jx_zs_60  l

In [ ]:
# Y_app2

In [ ]:
# y_pred2 = (Y_app2 > 0.5).type(torch.int32)
# y_pred2

In [ ]:
# model = TheModelClass(*args, **kwargs)
# optimizer = TheOptimizerClass(*args, **kwargs)

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']

数据准确性验证
过拟合/欠拟合
- 过拟合：对于已知数据过度拟合，对未知数据预测性差
- 欠拟合：对于已知数据拟合不够，对未知数据预测性差
  
tensor合并
- 

In [ ]:
# train_x.shape, train_y.shape
# train_cat = torch.cat([train_x, train_y],dim=1)   # 沿着Dimension 1进行合并
# train_cat.shape
# train_all.numpy()
# np.savetxt('../dataset/test20.csv', train_cat.numpy(),fmt='%.8f',delimiter=',') # 直接覆盖
# train_np = np.loadtxt('../dataset/test20.csv', dtype = np.float32, delimiter=',')

In [ ]:
# Ver 4.1 - random_split Mode， charlie
# from torch.utils.data import random_split    
# def accuracy(y_pred, y_true):
#     y_pred = (y_pred > 0.5).type(torch.int32)
#     acc = (y_pred == y_true).float().mean()
#     return acc

# HR_dataset = TensorDataset(X, Y)
# test_ds, test_ds = random_split(HR_dataset, [0.8, 0.2]) 
# HR_dataloader = DataLoader(test_ds, batch_size=batch, shuffle=True)  # shuffle乱序取数
# epochMode = 41
# if epochMode == 41:  
#     for epoch in range( epochs ):
#         # for x,y in HR_dataloader:
#         for x,y in HR_dataloader:
#             y_pred = model(x)
#             loss = loss_fn(y_pred, y)
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#         with torch.no_grad():
#             print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

Claude: 根据提供的notebook,Claude2对代码的建议和注释如下:
1. 数据处理部分可以添加更多注释,说明读取的数据字段含义,以及对数据的检查和处理思路。
2. 模型定义部分可以添加注释,说明输入和输出维度的设定依据,网络层数和神经元数量的选择依据等。
3. 训练评估部分的注释可以更详细,说明采用的数据分割方式,训练过程中的超参数,以及评估指标的计算方法。
4. 模型训练结果可以进行更深入的分析,检查是否存在过拟合或欠拟合,并结合业务目标提出后续优化思路。 
5. 可以添加模型效果可视化的代码,如Loss曲线,预测结果示例等,更直观地呈现模型效果。
6. 添加模型保存和加载的代码,便于训练后的模型重用。
7. 可以尝试不同模型结构或训练策略的比较实验。
8. 可以基于代码封装出函数或类,提高代码的复用性和模块化程度。
9. 添加类型注释,可以提高代码可读性和健壮性。
总体而言,该notebook内容完整,但仍有进一步提升注释、可视化、模块化、模型分析等方面的优化空间。